In [2]:
pip install torch


Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/198.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/198.5 MB 1.3 MB/s eta 0:02:36
   ---------------------------------------- 0.1/198.5 MB 1.3 MB/s eta 0:02:33
   ---------------------------------------- 0.2/198.5 MB 2.0 MB/s eta 0:01:42
   ---------------------------------------- 0.3/198.5 MB 2.1 MB/s eta 0:01:35
   ---------------------------------------- 0.6/198.5 MB 3.0 MB/s eta 0:01:07
   ---------------------------------------- 0.6/198.5 MB 2.5 MB/s eta 0:01:19
   ---------------------------------------- 0.7/198.5 MB 2.9 MB/s eta 0:01:09
   ---------------------------------------- 0.8/198.5 MB 2.5 MB/s eta 0:01:21
   ---------------------------------------- 0.8/198.5 MB 2.6 MB/s eta 0:01:18
   ---------------------------------------- 0.9/198.5 MB 2.1 MB/s eta 0:01:33
   ---------------------------------------- 1.2/198.5 MB 2.7 MB/s eta 0:01:13
   --


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install tensorboard

   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
    --------------------------------------- 0.1/5.5 MB 1.7 MB/s eta 0:00:04
   - -------------------------------------- 0.2/5.5 MB 2.3 MB/s eta 0:00:03
   -- ------------------------------------- 0.4/5.5 MB 2.2 MB/s eta 0:00:03
   --- ------------------------------------ 0.5/5.5 MB 2.7 MB/s eta 0:00:02
   ---- ----------------------------------- 0.6/5.5 MB 2.6 MB/s eta 0:00:02
   ------ --------------------------------- 0.8/5.5 MB 3.0 MB/s eta 0:00:02
   ------- -------------------------------- 1.1/5.5 MB 3.3 MB/s eta 0:00:02
   --------- ------------------------------ 1.3/5.5 MB 3.6 MB/s eta 0:00:02
   ----------- ---------------------------- 1.5/5.5 MB 3.9 MB/s eta 0:00:02
   ------------ --------------------------- 1.7/5.5 MB 3.9 MB/s eta 0:00:01
   -------------- ------------------------- 1.9/5.5 MB 4.0 MB/s eta 0:00:01
   --------------- -------


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install torchvision

SyntaxError: invalid syntax (1068751118.py, line 1)

In [5]:
#import libs
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torch.utils.tensorboard import SummaryWriter
import numpy as np

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
#normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [8]:
#SVHN dataset m22aie241 241 mod 3
train_dataset = datasets.SVHN(root="./data", split='train', transform=transform, download=True)
test_dataset = datasets.SVHN(root="./data", split='test', transform=transform, download=True)

 10%|▉         | 18186240/182040794 [01:40<11:38, 234740.70it/s]

In [ ]:
#train and test data loader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
#pretrained resnet 101
resnet101 = models.resnet101(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:04<00:00, 42.5MB/s]


In [ ]:
#freeze except final fully connected layer
for param in resnet101.parameters():
    param.requires_grad = False

In [ ]:
#SVHN classifiaction with 10 class
num_features = resnet101.fc.in_features
resnet101.fc = nn.Linear(num_features, 10)

In [ ]:
#train model
def train_model(optimizer_name, optimizer, model, criterion, train_loader, test_loader, device):
    model.to(device)
    model.train()
    writer = SummaryWriter(f'logs/{optimizer_name}')
    epochs = 10
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        train_loss = running_loss / len(train_loader)
        train_accuracy = 100 * correct / total
        writer.add_scalar('Loss/train', train_loss, epoch + 1)
        writer.add_scalar('Accuracy/train', train_accuracy, epoch + 1)
        print(f'Optimizer: {optimizer_name}, Epoch [{epoch+1}/{epochs}], Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}%')

#evaluate and test
    model.eval()
    top5_correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.topk(outputs, 5, dim=1)
            total += labels.size(0)
            for i in range(labels.size(0)):
                if labels[i] in predicted[i]:
                    top5_correct += 1
    test_accuracy_top5 = 100 * top5_correct / total
    print(f'Final {optimizer_name} Test Accuracy (Top-5): {test_accuracy_top5:.2f}%')
    writer.close()

In [ ]:
   # Save the trained model
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")

In [ ]:
#three optimizer adam adagrad and rms prop
optimizers = {
    'Adam': optim.Adam(resnet101.parameters(), lr=0.001),
    'Adagrad': optim.Adagrad(resnet101.parameters(), lr=0.01),
    'RMSprop': optim.RMSprop(resnet101.parameters(), lr=0.001)
}

In [ ]:
for optimizer_name, optimizer in optimizers.items():
    criterion = nn.CrossEntropyLoss()
    train_model(optimizer_name, optimizer, resnet101, criterion, train_loader, test_loader, device)

Optimizer: Adam, Epoch [1/10], Loss: 2.0060, Accuracy: 30.15%
Optimizer: Adam, Epoch [2/10], Loss: 1.9215, Accuracy: 33.71%
Optimizer: Adam, Epoch [3/10], Loss: 1.8873, Accuracy: 35.17%
Optimizer: Adam, Epoch [4/10], Loss: 1.8711, Accuracy: 35.62%
Optimizer: Adam, Epoch [5/10], Loss: 1.8638, Accuracy: 35.99%
Optimizer: Adam, Epoch [6/10], Loss: 1.8597, Accuracy: 36.46%
Optimizer: Adam, Epoch [7/10], Loss: 1.8540, Accuracy: 36.73%
Optimizer: Adam, Epoch [8/10], Loss: 1.8532, Accuracy: 36.51%
Optimizer: Adam, Epoch [9/10], Loss: 1.8530, Accuracy: 36.65%
Optimizer: Adam, Epoch [10/10], Loss: 1.8524, Accuracy: 36.64%
Final Adam Test Accuracy (Top-5): 82.74%
Optimizer: Adagrad, Epoch [1/10], Loss: 1.8377, Accuracy: 37.91%
Optimizer: Adagrad, Epoch [2/10], Loss: 1.7501, Accuracy: 39.61%
Optimizer: Adagrad, Epoch [3/10], Loss: 1.7440, Accuracy: 39.76%
Optimizer: Adagrad, Epoch [4/10], Loss: 1.7411, Accuracy: 40.07%
Optimizer: Adagrad, Epoch [5/10], Loss: 1.7390, Accuracy: 40.00%
Optimizer: Ad